In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy, pylab, scipy

import seaborn as sns
sns.set_context('talk') 
sns.set(font_scale=1.7)
sns.set_palette('colorblind')
sns.set_style('ticks')

pylab.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
    }
)

pylab.rcParams['axes.linewidth'] = 1
import warnings
warnings.filterwarnings("ignore", "Wswiglal-redir-stdio")
import lal

In [2]:
import os
from bilby.gw.conversion import luminosity_distance_to_redshift
from lal import SpinWeightedSphericalHarmonic
from scipy.interpolate import interp1d
from pycbc.types import TimeSeries as pycbc_timeseries
from pycbc.waveform.utils import taper_timeseries
from pycbc import pnutils
from astropy.constants import G, c, M_sun
def get_nr_signal(folder, mass_1_source, mass_2_source, luminosity_distance, inclination, phase, sampling_frequency,
                  mode_array = None, get_modes = False): 
    if not mode_array:
        mode_array = [[2,2], [2,1], [3,3], [3,2], [4,4],]   
    total_mass_source = mass_1_source + mass_2_source
    redshift = luminosity_distance_to_redshift(luminosity_distance)
    total_mass = total_mass_source * (1 + redshift)
    nr_modes = {}
    for mode in mode_array:
        print(f'Reading mode {mode}')
        file = os.path.join(folder, 'strain_l{}_m{}.dat'.format(mode[0], numpy.abs(mode[1])))
        data = numpy.loadtxt(file, skiprows=1, unpack=True)        
        times = total_mass * data[0] * G.value * M_sun.value / c.value **3
        hlm = pnutils.solar_mass_to_kg(total_mass) / pnutils.megaparsecs_to_meters(luminosity_distance) * (data[1] + 1j*data[2]) * G.value / c.value **2
        hlm_interped = interp1d(times, hlm)
        time_array = numpy.arange(times[0], times[-1], 1/sampling_frequency)
        hlm = hlm_interped(time_array)
        nr_modes[tuple(mode)] = {'time_array': time_array, 'hlm': hlm}
    
    if get_modes:
        return nr_modes
    signal = 0 + 0j 
    for mode in mode_array:
        print(f'Summing mode {mode}')
        Ylm = SpinWeightedSphericalHarmonic(inclination, phase, -2, mode[0], mode[1])
        signal += Ylm * nr_modes[tuple(mode)]['hlm']
        Yl_m = SpinWeightedSphericalHarmonic(inclination, phase, -2, mode[0], -mode[1])
        signal += Yl_m * (-1) ** mode[0] * numpy.conjugate(nr_modes[tuple(mode)]['hlm'])

    plus = taper_timeseries(pycbc_timeseries(numpy.real(signal), delta_t=1/sampling_frequency), tapermethod='TAPER_STARTEND')
    cross = taper_timeseries(pycbc_timeseries(numpy.imag(signal), delta_t=1/sampling_frequency), tapermethod='TAPER_STARTEND')

    return {'plus': plus, 'cross': cross}

def get_detector_response(ifo, injection_parameters):
    waveform_polarisations = get_nr_signal(folder=injection_parameters['folder'],
                                           mass_1_source=injection_parameters['mass_1_source'],
                                           mass_2_source=injection_parameters['mass_2_source'],
                                           luminosity_distance=injection_parameters['luminosity_distance'],
                                           inclination=injection_parameters['inclination'],
                                           phase=injection_parameters['phase'],
                                           sampling_frequency=ifo.sampling_frequency,
                                           )
           
    from pycbc.detector import add_detector_on_earth, Detector
    add_detector_on_earth(name=ifo.name,
                            longitude=ifo.longitude_radians,
                            latitude=ifo.latitude_radians,
                            yangle=numpy.deg2rad(360 - ifo.xarm_azimuth), 
                            xangle=numpy.deg2rad(360 - ifo.yarm_azimuth), 
                            height=0,
                            xlength=ifo.length * 1e3, 
                            ylength=ifo.length * 1e3) 
    detector = Detector(ifo.name)        
    waveform_polarisations['plus'] = taper_timeseries(waveform_polarisations['plus'], 'startend')
    waveform_polarisations['cross'] = taper_timeseries(waveform_polarisations['cross'], 'startend')    
    signal = detector.project_wave(waveform_polarisations['plus'], waveform_polarisations['cross'],
                                    injection_parameters['ra'], injection_parameters['dec'], injection_parameters['psi'], method='lal', reference_time = injection_parameters['geocent_time'])
    peak = abs(signal).numpy().argmax()
    snrp = signal[peak]
    time = signal.sample_times[peak]           
    signal.start_time += injection_parameters['geocent_time'] - time
    return signal
    
def inject(ifo, strain, injection_parameters):
    from pycbc.types import float64, float32
    import lalsimulation
    injection_func_map = {
        numpy.dtype(float32): lambda *args: lalsimulation.SimAddInjectionREAL4TimeSeries(*args),
        numpy.dtype(float64): lambda *args: lalsimulation.SimAddInjectionREAL8TimeSeries(*args),
    }

    signal = get_detector_response(ifo=ifo, injection_parameters=injection_parameters)
    signal = signal.astype(strain.dtype)
    lal_signal = signal.lal()
    lalstrain = strain.lal()
    add_injection = injection_func_map[strain.dtype]
    add_injection(lalstrain, lal_signal, None)
    strain.data[:] = lalstrain.data.data[:]
    return strain

In [ ]:
import h5py, pandas as pd
parameters = {}
with h5py.File('bns.h5', 'r') as f:
    for key in f.keys():
        parameters[key] = f[key][:]
        if len(parameters[key]) == 1:
            parameters[key] = numpy.zeros_like(f['total_mass'][:]) * parameters[key]
    df = pd.DataFrame(parameters)
df

In [ ]:
import bilby
from bilby.gw.detector.psd import PowerSpectralDensity as psd
minimum_frequency = 7
maximum_frequency = 4096

CE40 = bilby.gw.detector.Interferometer(name = "CE40",
                                        power_spectral_density = psd.from_amplitude_spectral_density_file('/ligo/home/ligo.org/koustav.chandra/projects/Cosmic-Explorer-MDC/gwforge/GWForge/ifo/noise_curves/CE40-asd.txt'),
                                        minimum_frequency = minimum_frequency,
                                        maximum_frequency = maximum_frequency,
                                        length = 40, 
                                        latitude = 46,
                                        longitude = -125,
                                        xarm_azimuth = 260,
                                        yarm_azimuth = 350,
                                        elevation = 0)

CE20 = bilby.gw.detector.Interferometer(name = "CE20",
                                        power_spectral_density =psd.from_amplitude_spectral_density_file('/ligo/home/ligo.org/koustav.chandra/projects/Cosmic-Explorer-MDC/gwforge/GWForge/ifo/noise_curves/CE20-asd.txt'),
                                        minimum_frequency = minimum_frequency,
                                        maximum_frequency = maximum_frequency,
                                        length = 40,
                                        latitude = 29,
                                        longitude = -94, 
                                        xarm_azimuth = 200,
                                        yarm_azimuth = 290,
                                        elevation = 0)


ifos = bilby.gw.detector.InterferometerList([CE40, CE20])
sampling_frequency = 2 * maximum_frequency
duration = 32
gps_start_time = 20
injection_parameters = {'folder' : 'EOSSFHo_M1309-1541_SR_MF/waveforms', 
                        'mass_1_source' : 1.541, 'mass_2_source': 1.309, 
                        'luminosity_distance' : 100,
                        'inclination' : 0,
                        'phase' : 0,
                        'geocent_time' : 1187008882.43,
                        'ra' : 197.450374,
                        'dec' : -23.381495,
                        'psi' : 0,
                        }

ifos.set_strain_data_from_power_spectral_densities(sampling_frequency=sampling_frequency,
                                                   duration=duration,
                                                   start_time=injection_parameters["geocent_time"] - gps_start_time)

In [ ]:
pylab.figure(figsize=(10,8))
for ifo in ifos:
    from gwpy.timeseries import TimeSeries
    strain = ifo.strain_data.to_pycbc_timeseries()
    data = inject(ifo=ifo, strain=strain, injection_parameters=injection_parameters)
    strain = TimeSeries.from_pycbc(data)
    peak = abs(data).numpy().argmax()
    time = data.sample_times[peak]                   
    time_delay = ifo.time_delay_from_geocenter(injection_parameters['ra'],
                                               injection_parameters['dec'],
                                               injection_parameters['geocent_time'])

    ifo.strain_data.set_from_time_domain_strain(time_domain_strain=data, start_time = injection_parameters['geocent_time'] + time_delay - gps_start_time, duration = duration, sampling_frequency = sampling_frequency)
    # data = get_detector_response(ifo, injection_parameters)
    pylab.plot(ifo.time_array - injection_parameters['geocent_time'] - time_delay, ifo.strain_data.time_domain_strain)
    pylab.xlim(-0.01,0.1)

from bilby.core.utils import infft
waveform_arguments = {'waveform_approximant' : 'NRSur7dq4',
                      'reference_frequency' : 11,
                      'catch_waveform_errors' : True,
                      'minimum_frequency' : 11}


waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,)

pylab.figure(figsize=(15,3))
gate_end = injection_parameters['geocent_time'] 
gate_start = gate_end  - 0.5
gated_likelihood = bilby.gw.likelihood.paint.GatedGaussianGravitationalWaveTransient(interferometers=ifos, 
                                                                                        waveform_generator=waveform_generator,
                                                                                        gate_start_time=gate_start,
                                                                                        gate_end_time=gate_end)
gated_likelihood.parameters = injection_parameters
gated = gated_likelihood.get_gated_painted_strain_data()
figure = pylab.figure(figsize=(8,8))
gs = figure.add_gridspec(3, 1, hspace=0.03)
k = 0
colors = {'CE40' : '#ca2000', 'CE20' : '#0571b0', 'V1' : '#171717'}
for (ifo, paint) in zip(ifos, gated):
    time_delay = ifo.time_delay_from_geocenter(injection_parameters['ra'],
                                               injection_parameters['dec'],
                                               injection_parameters['geocent_time'])
    tref = gate_end + time_delay

    ax = pylab.subplot(gs[k])
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(ifo.whitened_frequency_domain_strain,
                                                                 sampling_frequency=ifo.strain_data.sampling_frequency)
    ax.plot(ifo.time_array-tref, data_whitened_strain, color=colors[ifo.name], alpha=0.4, lw=2)  
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(paint.whitened_frequency_domain_strain,
                                                                    sampling_frequency=paint.strain_data.sampling_frequency)
    ax.plot(paint.time_array-tref, data_whitened_strain, color=colors[ifo.name], label=ifo.name, lw=2)    
    ax.set_xlim(-0.05, 0.025)
    strain = TimeSeries(ifo.strain_data.time_domain_strain, sample_rate=ifo.sampling_frequency,
            t0=ifo.strain_data.start_time, channel=f'{ifo.name}:INJ')
    file = os.path.join(f"{injection_parameters['folder']}", f"{ifo.name}.gwf")
    strain.write(file)
    k = k + 1

In [ ]:
injection_parameters

In [ ]:
waveform_polarisations = get_nr_signal(folder=injection_parameters['folder'],
                                        mass_1_source=injection_parameters['mass_1_source'],
                                        mass_2_source=injection_parameters['mass_2_source'],
                                        luminosity_distance=injection_parameters['luminosity_distance'],
                                        inclination=injection_parameters['inclination'],
                                        phase=injection_parameters['phase'],
                                        sampling_frequency=16384,
                                        )

pylab.figure(figsize=(15,3))
pylab.plot(waveform_polarisations['plus'].sample_times, waveform_polarisations['plus'].data)
pylab.plot(waveform_polarisations['cross'].sample_times, waveform_polarisations['cross'].data)

In [ ]:
frequency_domain_strain = {'plus': waveform_polarisations['plus'].to_frequencyseries(), 'cross': waveform_polarisations['cross'].to_frequencyseries()}
amplitude = (frequency_domain_strain['plus'].squared_norm() + frequency_domain_strain['cross'].squared_norm())**0.5
amplitude *= numpy.sqrt(frequency_domain_strain['plus'].sample_frequencies)
pylab.loglog(frequency_domain_strain['plus'].sample_frequencies, amplitude)


In [ ]:
pylab.figure(figsize=(10,8))
for ifo in ifos:
    pylab.loglog(ifo.frequency_array, ifo.freq)

In [ ]:
from bilby.core.utils import infft
waveform_arguments = {'waveform_approximant' : 'NRSur7dq4',
                      'reference_frequency' : 11,
                      'catch_waveform_errors' : True,
                      'minimum_frequency' : 11}


waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,)

pylab.figure(figsize=(15,3))
gate_end = injection_parameters['geocent_time'] 
gate_start = gate_end  - 0.5
gated_likelihood = bilby.gw.likelihood.paint.GatedGaussianGravitationalWaveTransient(interferometers=ifos, 
                                                                                        waveform_generator=waveform_generator,
                                                                                        gate_start_time=gate_start,
                                                                                        gate_end_time=gate_end)
gated_likelihood.parameters = injection_parameters
gated = gated_likelihood.get_gated_painted_strain_data()
figure = pylab.figure(figsize=(8,8))
gs = figure.add_gridspec(3, 1, hspace=0.03)
k = 0
colors = {'CE40' : '#ca2000', 'CE20' : '#0571b0', 'V1' : '#171717'}
for (ifo, paint) in zip(ifos, gated):
    time_delay = ifo.time_delay_from_geocenter(injection_parameters['ra'],
                                               injection_parameters['dec'],
                                               injection_parameters['geocent_time'])
    tref = gate_end + time_delay

    ax = pylab.subplot(gs[k])
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(ifo.whitened_frequency_domain_strain,
                                                                 sampling_frequency=ifo.strain_data.sampling_frequency)
    ax.plot(ifo.time_array-tref, data_whitened_strain, color=colors[ifo.name], alpha=0.4, lw=2)  
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(paint.whitened_frequency_domain_strain,
                                                                    sampling_frequency=paint.strain_data.sampling_frequency)
    ax.plot(paint.time_array-tref, data_whitened_strain, color=colors[ifo.name], label=ifo.name, lw=2)    
    k = k + 1
         

# for (ifo, paint) in zip(ifos, gated):
#     time_delay = ifo.time_delay_from_geocenter(injection_parameters['ra'],
#                                             injection_parameters['dec'],
#                                             injection_parameters['geocent_time'])
#     tref = gate_end + time_delay #+ 0.004

#     ax = pylab.subplot(gs[k])

#     data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(ifo.whitened_frequency_domain_strain,
#                                                                 sampling_frequency=ifo.strain_data.sampling_frequency)
#     ax.plot(ifo.time_array-tref, data_whitened_strain, color=colors[ifo.name], alpha=0.4, lw=2)  
#     data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(paint.whitened_frequency_domain_strain,
#                                                                 sampling_frequency=paint.strain_data.sampling_frequency)
#     ax.plot(paint.time_array-tref, data_whitened_strain, color=colors[ifo.name], label=ifo.name, lw=2)
    ax.set_xlim(-0.05, 0.025)
#     # ax.set_ylim(-5,5)
#     # ax.set_xticks([-0.1, -0.05, 0, 0.05,0.1,0.14])
#     ax.grid(linestyle=':', color='#171717')

In [ ]:
for ifo in ifos:
    strain = ifo.strain_data.to_gwpy_timeseries()
    strain.write('')


In [ ]:
time - gate_end

In [ ]:
frames = {'CE40' : '/ligo/home/ligo.org/koustav.chandra/projects/bilby-qnm/xg/bns-post-merger/EOSSFHo_M1309-1541_SR_MF/waveforms/CE40.gwf',
          'CE20' : '/ligo/home/ligo.org/koustav.chandra/projects/bilby-qnm/xg/bns-post-merger/EOSSFHo_M1309-1541_SR_MF/waveforms/CE20.gwf'}

duration = 20
for ifo in ifos:
    ifo.set_strain_data_from_frame_file(frame_file=frames[ifo.name], 
                                        sampling_frequency=sampling_frequency, 
                                        duration=duration,
                                        start_time = injection_parameters['geocent_time'] - 10,
                                        channel=f'{ifo.name}:INJ')
    ifo.minimum_frequency = minimum_frequency
    ifo.maximum_frequency = maximum_frequency


from bilby.core.utils import infft
waveform_arguments = {'waveform_approximant' : 'NRSur7dq4',
                      'reference_frequency' : 11,
                      'catch_waveform_errors' : True,
                      'minimum_frequency' : 11}


waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_black_hole,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_black_hole_parameters,
    waveform_arguments=waveform_arguments,)

pylab.figure(figsize=(15,3))
gate_end = injection_parameters['geocent_time'] 
gate_start = gate_end  - 0.5
gated_likelihood = bilby.gw.likelihood.paint.GatedGaussianGravitationalWaveTransient(interferometers=ifos, 
                                                                                        waveform_generator=waveform_generator,
                                                                                        gate_start_time=gate_start,
                                                                                        gate_end_time=gate_end)
gated_likelihood.parameters = injection_parameters
gated = gated_likelihood.get_gated_painted_strain_data()
figure = pylab.figure(figsize=(8,8))
gs = figure.add_gridspec(3, 1, hspace=0.03)
k = 0
colors = {'CE40' : '#ca2000', 'CE20' : '#0571b0', 'V1' : '#171717'}
for (ifo, paint) in zip(ifos, gated):
    time_delay = ifo.time_delay_from_geocenter(injection_parameters['ra'],
                                               injection_parameters['dec'],
                                               injection_parameters['geocent_time'])
    tref = gate_end + time_delay

    ax = pylab.subplot(gs[k])
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(ifo.whitened_frequency_domain_strain,
                                                                 sampling_frequency=ifo.strain_data.sampling_frequency)
    ax.plot(ifo.time_array-tref, data_whitened_strain, color=colors[ifo.name], alpha=0.4, lw=2)  
    data_whitened_strain = (1./(4.*numpy.pi * ifo.duration))*infft(paint.whitened_frequency_domain_strain,
                                                                    sampling_frequency=paint.strain_data.sampling_frequency)
    ax.plot(paint.time_array-tref, data_whitened_strain, color=colors[ifo.name], label=ifo.name, lw=2)    
    k = k + 1

In [ ]:
pylab.plot(ifo.time_array, ifo.time_domain_strain)

In [ ]:
# Create submit files 
template='''######################################################################
# Submit Script for bilby ringdown
######################################################################
universe = vanilla
executable = {}/bns.py
arguments = "--toffset {}"
log = {}/logs/{}.log
error = {}/logs/{}.err
output = {}/logs/{}.out
accounting_group = ligo.dev.o4.cbc.pe.lalinference
notification = NEVER
request_cpus = 1
getenv = True
request_disk = 100MB
priority = 1000
machine_count = 1
+InitialRequestMemory = 5000
request_memory = ifthenelse( (LastHoldReasonCode=!=21 && LastHoldReasonCode=!=26 && LastHoldReasonCode=!=34), InitialRequestMemory, int(1.5 * MemoryUsage) )
periodic_release = ((HoldReasonCode =?= 21) || (HoldReasonCode =?= 26) || (HoldReasonCode =?= 34))
periodic_remove = (JobStatus == 5) && ((CurrentTime - EnteredCurrentStatus) > 43200)
queue 
'''

from pycbc.conversions import final_mass_from_initial
import numpy, bilby, os
from astropy.constants import G, c, M_sun

final_mass = 0.95 * (1.309 + 1.541)

kerr = 'XAS'
outdir = '/ligo/home/ligo.org/koustav.chandra/projects/bilby-qnm/xg/bns-post-merger'
dag = os.path.join(outdir, f'{kerr}.condor')
with open(dag, 'w') as f:     
    for toffset in [0, 10, 12]:
        logs = os.path.join(outdir, 'logs')
        if not os.path.exists(logs):
            os.makedirs(logs)
        name = '{}-{}M'.format(kerr, toffset)
        submit_file = os.path.join(outdir, name+'.sub')
        with open(submit_file, 'w') as fo:
            fo.write(template.format(outdir, # path to executable
                                     toffset, # toffset
                                     outdir,'{}-{}M'.format(kerr, toffset), # logs
                                     outdir,'{}-{}M'.format(kerr, toffset), # err
                                     outdir,'{}-{}M'.format(kerr, toffset), # out
                                     ))                                        
        f.write(f'JOB {name} {submit_file} \n')

In [ ]:
bil